In [1]:
# libraries
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import random
import time
import gspread

In [20]:
# set up dataframes & lists for indexing
info_cols = ['link','address','beds','baths','sqft']
history_cols = ['address','Date','Event','Price']

# history_df = pd.DataFrame(columns = history_cols)
# info_df = pd.DataFrame(columns = info_cols)
history_df = pd.read_csv('historical_initial.csv')
info_df = pd.read_csv('info_initial.csv')


props = ['og:zillow_fb:address','zillow_fb:beds','zillow_fb:baths']
labels = ['address','beds','baths','sqft','link']

In [5]:
# # get initial list of houses and pages to get more
# houses = list(pd.read_csv('/Users/lelee1/Desktop/D590TS/homes.csv')['0'])
# zip_links = list(pd.read_csv('/Users/lelee1/Desktop/D590TS/addresspages.csv')['0'])
# random.shuffle(zip_links)

In [ ]:
# # get links for more houses
# def homedetails(href):
#     return href and re.compile("/homedetails").search(href)


# for url in zip_links:
#     browser = webdriver.Chrome(executable_path='/Users/lelee1/Desktop/D590TS/chromedriver')
#     browser.get(url)
#     soup = BeautifulSoup(browser.page_source)
#     time.sleep(random.sample(list(range(3)),1)[0])
#     browser.close()
#     county_homes = []
#     for a in soup.find_all(href = homedetails):
#         county_homes.append('https://www.zillow.com' + a.get('href'))
#     if len(county_homes) < 200:
#         n = len(county_homes)
#     else:
#         n = 200
#     houses = houses + random.sample(county_homes, n)
#     zip_links.remove(url)


In [43]:
# # update the houses list and the zip links list for later
# homedf = pd.DataFrame(houses)
# homedf.to_csv('homes.csv')

# zipdf = pd.DataFrame(zip_links)
# zipdf.to_csv('addresspages.csv')

In [23]:
home_links = list(pd.read_csv('homes.csv')['0'])
random.shuffle(home_links)
# homedf = pd.DataFrame(home_links)
# homedf.to_csv('homes.csv')

In [24]:
len(home_links)


17123

In [ ]:
for url in home_links:
    browser = webdriver.Chrome(executable_path='/Users/lelee1/Desktop/D590TS/chromedriver')
    browser.get(url)
    soup = BeautifulSoup(browser.page_source)
    browser.close()
    try:
        for x in soup.find_all('meta', {'property':"og:zillow_fb:address"},'content'):
            address = x.get('content')


        concat_events = [tr.get('label') for tr in soup.find_all('tr') if tr.get('label') is not None]

        for row in concat_events:
            new = []
            new_row = row.split(', ')
            for x in new_row:
                if ':' in x:
                    new.append(map(str.strip,x.split(':',1)))
            new = dict(new)
            new['address'] = address
            history_df = history_df.append(new, ignore_index = True)
    
        info = []
        for p in props:
            for x in soup.find_all('meta',{'property':p},'content'):
                info.append(x.get('content'))
        info.append(''.join(re.findall('([0-9])',soup.find_all('span',{'data-testid':"bed-bath-beyond"})[0].find_all('span')[6].text)))
        info.append(url)
        info_new = {labels[i]:info[i] for i in range(len(labels))}
        info_df = info_df.append(info_new, ignore_index=True)
    except:
        pass
    home_links.remove(url)

In [10]:
info_df

,Unnamed: 0,Unnamed: 0.1,link,address,list_price,beds,baths,sqft
0,0.0,0.0,https://www.zillow.com/homedetails/100-W-Wabas...,"100 W Wabash St, Berne, IN 46711",NaN,2,0.0,1080.0
1,1.0,1.0,https://www.zillow.com/homedetails/1002-Dearbo...,"1002 Dearborn St, Berne, IN 46711",NaN,3,1.5,1700.0
2,2.0,2.0,https://www.zillow.com/homedetails/1005-Dearbo...,"1005 Dearborn St, Berne, IN 46711",NaN,3,2.0,1746.0
3,3.0,3.0,https://www.zillow.com/homedetails/1005-E-Main...,"1005 E Main St, Berne, IN 46711",NaN,0,0.0,4500.0
4,4.0,4.0,https://www.zillow.com/homedetails/1005-Mill-P...,"1005 Mill Pond Rd, Berne, IN 46711",NaN,3,3.0,1548.0
...,...,...,...,...,...,...,...,...
3713,NaN,NaN,https://www.zillow.com/homedetails/810-Park-Av...,"810 Park Ave, South Bend, IN 46616",NaN,3,2,1488
3714,NaN,NaN,https://www.zillow.com/homedetails/50-N-Hamilt...,"50 N Hamilton St, Pittsboro, IN 46167",NaN,2,1,754
3715,NaN,NaN,https://www.zillow.com/homedetails/3540-E-Debb...,"3540 E Debby Dr, Knox, IN 46534",NaN,3,1.5,2070
3716,NaN,NaN,https://www.zillow.com/homedetails/7049-Kingsw...,"7049 Kingswood Cir, Indianapolis, IN 46256",NaN,0,3,2449


In [11]:
# dataframes to csv
history_df.to_csv('historical_initial.csv', index = False)
info_df.to_csv('info_initial.csv', index = False)